# Murphy Angelo & Cyrus Tau project report

### Detecting riboswitches with diffusion + SOFM

RNA flexibility is a key challenge in predicting RNA structures and is the defining characteristic of riboswitches, which show promise as selective gene regulation platforms. We used a diffusion model to generate 5000 structures for 4053 riboswitch sequences and 4053 shuffled controls. For each of the ~8000 sequences, we applied SOFMs to analyze the 5000 structures (represented as LxL binary tensors) to quantify alternative conformations.  We used a simple logistic regressor trained on these features to achieve state-of-the-art riboswitch detection vs. established methods.

In [1]:
import os
import matplotlib
import networkx as nx
import numpy as np
from joblib import Parallel, delayed
from karateclub import Graph2Vec
from pathlib import Path

In [ ]:
import ct_tools as ct
ct.setup_figure_saving('murfalo_cyrustau_project')

In [2]:
def load_x0(prediction: Path):
    """
    Loads predictions as NxLxL matrices. N := number of diffusion samples.
    L := sequence length.
    """
    out_file = prediction / "x0_compressed.npz"
    with np.load(out_file) as data:
        x0, seq = data["x0"], data["seq"]
    return seq, x0

In [3]:
pred_dir = Path("./predict_results")
predictions = list(
    path for path in pred_dir.iterdir() if not path.name.endswith("_shuffled")
)

### Prediction suffix naming rules
- "" => normal sequence
- "_shuffled" => mononucleotide shuffle
- "_shuffled_dinuc" => dinucleotide shuffle (preserves dinuc. frequencies)

In [4]:
# Calculate graph2vec embeddings of RNA adjacency matrices
# NOTE(MCA): Very slow, better to run on HPC
ignore_cache = False
if ignore_cache == True:

    def compute_embedding(prediction):
        model = Graph2Vec(dimensions=128, workers=1)
        _, x0 = load_x0(prediction)  # (N, L, L)
        # x0 = (x0 + x0.transpose(0, 2, 1)) / 2.0  # enforce symmetry
        graphs = [nx.from_numpy_array(a) for a in x0]

        model.fit(graphs)
        return model.get_embedding()  # (N, d)

    embeddings = Parallel(n_jobs=-1, backend="loky")(
        delayed(compute_embedding)(p) for p in predictions
    )
    embeddings = np.stack(embeddings)  # (P, N, d)
    for prediction, embedding in zip(predictions, embeddings):
        np.savez_compressed(
            prediction / "g2v.npz", embedding=embedding.astype(np.float32)
        )


def load_g2v(path: Path) -> np.ndarray:
    return np.load(path / "g2v.npz")["embedding"]


# Load embeddings
embeddings = np.stack(
    Parallel(n_jobs=-1, backend="loky")(delayed(load_g2v)(p) for p in predictions)
)

embeddings = np.stack([np.load(p / "g2v.npz")["embedding"] for p in predictions])

In [71]:
from minisom import MiniSom


def som(prediction, embedding) -> MiniSom:
    # SOM hyperparameters
    m_neurons = 2
    n_neurons = 2
    sigma = (m_neurons / n_neurons) / 2.0
    eta = 0.5

    # Generate SOMs for each RNA
    N, L = embedding.shape
    som = MiniSom(
        m_neurons,
        n_neurons,
        L,
        sigma=sigma,
        learning_rate=eta,
        neighborhood_function="gaussian",
        topology="rectangular",
        random_seed=42,
    )
    som.pca_weights_init(embedding)
    som.train(embs, N, verbose=False)
    return som


soms = list(
    Parallel(n_jobs=-1, backend="loky")(
        delayed(som)(p, emb) for p, emb in zip(predictions, embeddings)
    )
)

In [72]:
# Plot SOM U matrices
import matplotlib.pyplot as plt

%matplotlib inline

som, embs = soms[1], embeddings[1]
w_x, w_y = zip(*[som.winner(d) for d in embs])
w_x = np.array(w_x)
w_y = np.array(w_y)

plt.figure(figsize=(10, 9))
plt.pcolor(som.distance_map().T, cmap="bone_r", alpha=0.2)
plt.colorbar()
plt.scatter(
    w_x + 0.5 + (np.random.rand(len(w_x)) - 0.5) * 0.8,
    w_y + 0.5 + (np.random.rand(len(w_y)) - 0.5) * 0.8,
    s=50,
    c="C0",
)

plt.legend(loc="upper right")
plt.grid()
plt.show()

/var/folders/5d/hx6lx2b96_q3ty9csdybv1440000gn/T/ipykernel_12569/1198385565.py:20: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(loc='upper right')


<Figure size 1000x900 with 2 Axes>

In [93]:
# Create feature vector and labels for training


def get_features(prediction, som, embs):
    """
    Returns X, y (features and labels) for a given prediction, som, and embeddings.
    """
    _, x0 = load_x0(prediction)  # (N, L, L)
    p_bp = x0.max(axis=1).mean(axis=(0, 1))  # avg. base pair probability
    qe = som.quantization_error(embs)
    return (x0_frob, p_bp, qe, prediction.name.endswith("_shuffled_dinuc"))


features = np.stack(
    Parallel(n_jobs=-1, backend="loky")(
        delayed(get_features)(p, som, emb)
        for p, som, emb in zip(predictions, soms, embeddings)
    )
)
X, y = features[:, :-1], features[:, -1]
del features

/opt/homebrew/Caskroom/miniforge/base/envs/9530/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [95]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    precision_recall_curve,
    roc_auc_score,
    balanced_accuracy_score,
    matthews_corrcoef,
)

# Simple logistic regressor with LOOCV across families
accessions, families = zip(*[p.name.split("_")[0:2] for p in predictions])
accessions, families = np.stack(accessions), np.stack(families)
family_to_accession = {
    family: accession for family, accession in zip(families, accessions)
}

unique_families = sorted(set(families))
family_metrics = []
for family in unique_families:
    print(f"Predicting {family}...")
    # Split data into training and test sets
    test_mask = families == family
    train_mask = ~test_mask
    X_train = X[train_mask]
    y_train = y[train_mask]
    X_test = X[test_mask]
    y_test = y[test_mask]

    # Skip empty families (only possible if all runs are not finished)
    if len(np.unique(y_test)) < 2:
        continue

    # Standardize features: fit scaler on training data only to avoid leakage
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    # Hyperparameter search
    param_grid = {
        "C": [0.01, 0.1, 1, 10],
        "penalty": ["elasticnet"],
        "solver": ["saga"],
        "l1_ratio": [0.0, 0.2, 0.5, 0.8, 1.0],  # 0=L2, 1=L1
    }
    clf = GridSearchCV(
        LogisticRegression(max_iter=10000),
        param_grid,
        cv=5,
        scoring="roc_auc",
        n_jobs=-1,
    )
    clf.fit(X_train_scaled, y_train)
    best_clf = clf.best_estimator_

    # Predictions and scores
    y_scores_test = best_clf.predict_proba(X_test_scaled)[:, 1]
    y_pred_test = best_clf.predict(X_test_scaled)

    # Compute and append metrics
    with np.errstate(divide="ignore", invalid="ignore"):
        precision, recall, _ = precision_recall_curve(y_test, y_scores_test)
        F1_scores = np.nan_to_num(2 * (precision * recall) / (precision + recall))
        max_f1 = np.max(F1_scores)
        auc = roc_auc_score(y_test, y_scores_test)
        balanced_acc = balanced_accuracy_score(y_test, y_pred_test)
        mcc = matthews_corrcoef(y_test, y_pred_test)

        # Compute AUC
        family_metrics.append(
            {
                "accession": family_to_accession[family],
                "family": family,
                "num_samples": np.sum(test_mask),
                "max_f1": max_f1,
                "auc": auc,
                "balanced_acc": balanced_acc,
                "mcc": mcc,
                "avg_length": np.mean(X[test_mask, -1]),
            }
        )

# Compute and print average metrics across all families
avg_max_f1 = np.mean([m["max_f1"] for m in family_metrics])
avg_auc = np.mean([m["auc"] for m in family_metrics])
print(f"Cross-validated F1: {avg_max_f1:.8f}")
print(f"Cross-validated AUC: {avg_auc:.8f}")

# Interpretable decision boundary on all unscaled data
if X.shape[1] == 2 and len(np.unique(y)) > 1:
    param_grid = {
        "C": [0.01, 0.1, 1, 10],
        "penalty": ["elasticnet"],
        "solver": ["saga"],
        "l1_ratio": [0.0, 0.2, 0.5, 0.8, 1.0],  # 0=L2, 1=L1
    }
    final_clf_search = GridSearchCV(
        LogisticRegression(max_iter=10000),
        param_grid,
        cv=5,
        scoring="roc_auc",
        n_jobs=-1,
    )
    final_clf_search.fit(X, y)
    final_clf = final_clf_search.best_estimator_
    # FIXME(MCA): Restore feature labels
    # plot_fig(
    #     X,
    #     y,
    #     ["Mean Contact Strength", "Ensemble Concentration"],
    #     coef=final_clf.coef_,
    #     intercept=final_clf.intercept_,
    # )

Predicting AdoCbl...


/opt/homebrew/Caskroom/miniforge/base/envs/9530/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Predicting Cobalamin...
Predicting FMN...
Predicting Fluoride...
Predicting Glycine...
Predicting Lysine...
Predicting MFR...
Predicting MOCO...
Predicting NiCo...
Predicting PreQ1...
Predicting PreQ1-III...
Predicting Purine...
Predicting SAH...
Predicting SAM...
Predicting SAM-I-IV-variant...
Predicting SAM-IV...
Predicting SAM-SAH...
Predicting SMK...
Predicting THF...
Predicting TPP...
Predicting ZMP-ZTP...
Predicting c-di-GMP-I...
Predicting c-di-GMP-II...
Predicting glnA...
Predicting preQ1-II...
Cross-validated F1: 0.78283041
Cross-validated AUC: 0.77944846


In [96]:
# Compare Goodarzi performance vs. ours
# --- Plot styling ---
plt.rcParams["font.size"] = 16
plt.rcParams["axes.titlesize"] = 24
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["legend.fontsize"] = 16

# --- Load data ---
df_mine = pd.DataFrame(family_metrics)
df_goodarzi = pd.read_csv("goodarzi_metrics.csv")

# --- Process data ---
overlapping_families = list(
    set(df_mine["accession"]) & set(df_goodarzi["Rfam family identifier"])
)
overlapping_families.sort()
auc_mine = [
    df_mine[df_mine["accession"] == family]["auc"].values[0]
    for family in overlapping_families
]
auc_goodarzi = [
    df_goodarzi[df_goodarzi["Rfam family identifier"] == family]["AUC value"].values[0]
    for family in overlapping_families
]
n_families = len(overlapping_families)

# --- Plot the data ---
fig, ax = plt.subplots(figsize=(8, 6))
for i in range(n_families):
    color = "black" if auc_mine[i] > auc_goodarzi[i] else "#bf616a"
    ax.plot(
        [0.4, 1.6],
        [auc_goodarzi[i], auc_mine[i]],
        color=color,
        alpha=0.7,
        linestyle="dotted",
        linewidth=1.5,
        zorder=1,
    )

# Scatter
ax.scatter(
    [0.4] * n_families,
    auc_goodarzi,
    color="#8be9fd",
    s=100,
    label="Goodarzi",
    alpha=0.9,
    edgecolor="k",
    linewidths=2,
    zorder=2,
)
ax.scatter(
    [1.6] * n_families,
    auc_mine,
    color="#ffb86c",
    s=100,
    label="Ours",
    alpha=0.9,
    edgecolor="k",
    linewidths=2,
    zorder=2,
)

# Axes
ax.set_xticks([0.4, 1.6])
ax.set_xticklabels(["Goodarzi", "Ours"], fontweight="bold", fontsize=14)
ax.set_xlim(0, 2)
ax.set_ylabel("AUC", fontweight="bold", fontsize=14)
ax.set_ylim(0, 1)

# Title
ax.set_title("Goodarzi vs. Ours", fontweight="bold", pad=25, fontsize=18)

# Background
ax.set_facecolor("#f5f5f5")
ax.grid(True, axis="y", linestyle="--", alpha=0.5, color="gray")

# Border
for spine in ax.spines.values():
    spine.set_edgecolor("black")
    spine.set_linewidth = 2

# Layout + save
plt.tight_layout()
plt.savefig("Versus Goodarzi.png", dpi=300, bbox_inches="tight")

<Figure size 800x600 with 1 Axes>